In [3]:
#%%
import vedo
import napari
import numpy as np
from pygeodesic import geodesic
import napari_process_points_and_surfaces as nppas
from astropy.convolution import Gaussian2DKernel, convolve,interpolate_replace_nans
import os
import pandas as pd
import watershed_udder as wu
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pyrealsense2 as rs

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\marie\anaconda3\envs\napari\lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
  File "C:\Users\marie\AppData\Local\Temp\ipykernel_3584\736828575.py", line 6, in <module>
    import napari_process_points_and_surfaces as nppas
  File "C:\Users\marie\anaconda3\envs\napari\lib\site-packages\napari_process_points_and_surfaces\__init__.py", line 11, in <module>
    import stackview
  File "C:\Users\marie\anaconda3\envs\napari\lib\site-packages\stackview\__init__.py", line 6, in <module>
    from ._crop import crop
  File "C:\Users\marie\anaconda3\envs\napari\lib\site-packages\stackview\_crop.py", line 1, in <module>
    from ipywidgets import VBox
ModuleNotFoundError: No module named 'ipywidgets'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\marie\anaconda3\envs\napari\lib\site-packages\pygments\styles\__init__.py", line 89, in get_style_by

In [ ]:
def points_toworld(points):
    points2 = points.copy()
    for i in range(len(points)):
        points2[i, :] = rs.rs2_deproject_pixel_to_point(intr, [points[i, 0], points[i, 1]], points[i, 2])
    return points2

In [2]:

dirpath = os.getcwd()
ws_dir = r"validate_watershed\watershed_segments"
corr_dir = r"validate_watershed\watershed_correspondence"
label_dir = os.path.join(dirpath, r"validate_watershed\pred_labels")
kp_dir = os.path.join(label_dir, r"keypoints")
sg_dir = os.path.join(label_dir, r"segments")
img_dir = os.path.join(os.path.normpath(dirpath + os.sep + os.pardir), r"udder_video\depth_images")
results = pd.read_csv(r"validate_watershed\ws_class_predictions_I.csv")
good = results[results.thr09 == 1]
filenames = [file.replace(".npy", "") for file in os.listdir(ws_dir)]
cows = set()
filenames2 = []
for file in filenames: 
    cow = file.split("_")[0]
    if cow not in cows:
        cows.add(cow)
        filenames2.append(file)

video_path =  os.path.join(os.path.normpath(dirpath + os.sep + os.pardir), r"udder_video\video_files\example_video.bag")

results_df = pd.DataFrame(columns = ["cow", "filename", "volume", "lf_vol", "rf_vol", "lb_vol", "rb_vol"])

config = rs.config()
rs.config.enable_device_from_file(config, video_path, repeat_playback = False)
pipeline = rs.pipeline()
cfg = pipeline.start(config) # Start pipeline and get the configuration it found
profile = cfg.get_stream(rs.stream.depth) # Fetch stream profile for depth stream
intr = profile.as_video_stream_profile().get_intrinsics() # Downcast to video_stream_profile and fetch intrinsics
# depth_sensor = profile.as_video_stream_profile().get_device().first_depth_sensor()
# depth_scale = depth_sensor.get_depth_scale()

# quarter assignment - according to ws_map 
# lf - yelow (255,255,0)
# rf - cian (0, 255, 255)
# lb - magenta (255, 0, 255)
# rb - white (255,255,255)
# background - black
color_dict = {"lf":[1,1,0], "rf": [0, 1, 1], "lb":[1, 0,1], "rb":[0.5,0.5,0.5], "bg": [0, 0, 0]}

for file in good.filename[:1]:
    cow = file.split("_")[0]
    cow_line = {"cow": cow, "filename":file, "volume": np.nan, "lf_vol":np.nan, "rf_vol": np.nan, "lb_vol":np.nan, "rb_vol":np.nan}
    # udder object
    udder = wu.udder_object(file + ".tif", img_dir, label_dir, array = 0)
    # read image
    img = udder.img
    # read labels
    segment = udder.get_segment()
    points = udder.get_keypoints()
    # reas WS segmentation
    ws_label = np.load(os.path.join(ws_dir, file + ".npy"))
    kp_ws = pd.read_csv(os.path.join(corr_dir, file +".csv")).loc[0].to_dict()
    ws_map = dict((v, k) for k, v in kp_ws.items())
    ws_map[0] = "bg"
    new_kp = wu.update_kp(kp_ws, ws_label, img)
    
    scale = 0.001
    img = udder.img.copy().astype(float)
    img[img ==0] = np.nan
    kernel = Gaussian2DKernel(x_stddev=1)
    udder_conv = convolve(img, kernel)
    udder_conv[np.isnan(udder_conv)] = 0
    
    masked_udder =  udder.get_mask() * udder_conv 
    rows, cols = np.nonzero(masked_udder)
    values = masked_udder[rows, cols]
    quarter_lbls = ws_label[rows, cols]
    quarter_colors = quarter_colors = np.array([color_dict[ws_map[point]] if point in ws_map.keys() else [0,0,0]for point in quarter_lbls ])
    udder_points = np.column_stack((np.transpose(cols), np.transpose(rows), np.transpose(values))).astype(float)
    udder_points[:, 2] = udder_points[:, 2] *scale
    pts = points_toworld(udder_points)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\marie\anaconda3\envs\napari\lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
  File "C:\Users\marie\AppData\Local\Temp\ipykernel_3584\2811628374.py", line 1, in <module>
    dirpath = os.getcwd()
NameError: name 'os' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\marie\anaconda3\envs\napari\lib\site-packages\pygments\styles\__init__.py", line 89, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\marie\anaconda3\envs\napari\lib\site-packages\IPython\core\interactiveshell.py", line 2144, in showtraceback
  File "C:\Users\marie\anaconda3\envs\napari\lib\site-packages\IPython\core\ultratb.py", line 1435, in structured_traceback
  File "C:\Users\marie\anaconda3\envs\napar

In [ ]:
viewer = napari.Viewer(ndisplay=3)

Cannot move to target thread (0x1fe3be82630)



In [1]:
hide_all(viewer)

viewer.add_surface(nppas.create_convex_hull_from_points(centroids))

napari.utils.nbscreenshot(viewer)

NameError: name 'hide_all' is not defined